In [1]:
import os
os.chdir('../')
%pwd

'd:\\Programming\\DataScienceProjects\\mlflow-project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    training_data: Path
    testing_data: Path
    model: str
    alpha: float
    l1_ratio: float
    target_column: str

In [3]:
from mlflowProject.constants import *
from mlflowProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath =  PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> ModelTrainerConfig:
        config = self.config
        params = self.params
        schema = self.schema

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            training_data = config.training_data,
            testing_data = config.testing_data,
            model = config.model,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.target_column
        )

        return model_trainer_config